# TP4  
Félix Adam  
Charles-Antoine Lanoix

Note: Utiliser la fonction tree.postorderTrav() pour afficher les arbres

In [202]:
import numpy as np

# Fonctions outils  



In [203]:
#Prend une liste de listes et remet tous les éléments dans une seule liste
def flattenList(l):
    outList = []
    for i in range(len(l)):
        if(l[i]==None):
            continue
        for j in range(len(l[i])):
            if(l[i][j] != None):
                outList.append(l[i][j])
    return outList

In [204]:
#Prend une matrice de distance et calcule la nouvelle en fusionnant les noeud i et j
def getNewMatrix(matrix,i,j):
    
    #trouver les index de la matrices qui ne sont pas i et j
    remainingIndexes = []
    for k in range(len(matrix)):
        if(k != i and k != j):
            remainingIndexes.append(k)
          
    #nouvelle matrice de dimension plus petite
    newM = np.diag(np.full((len(matrix)-1,len(matrix)-1),float("inf")).diagonal())
    
    #réinsérer les distances de l'ancienne matrice dans la nouvelle
    #(pour les index encore présents)
    for n in range(len(remainingIndexes)):
        for m in range(n+1,len(remainingIndexes)):
            #print(remainingIndexes[n],remainingIndexes[m])
            newM[n,m] = matrix[remainingIndexes[n]][remainingIndexes[m]]
            newM[m,n] = newM[n,m]
        #print(matrix[i][remainingIndexes[n]],matrix[j][remainingIndexes[n]],matrix[i][j])[:-1]
        #ajouter les nouvelles valeurs calculées dans la dernière colonne/ligne
        newM[n,-1] = 1/2*(matrix[i][remainingIndexes[n]]+matrix[j][remainingIndexes[n]]-matrix[i][j])
        newM[-1,n] = newM[n,-1]
        
    return newM

In [205]:
#Test de la dernière fonction
matrix = [[float("inf"),1,2,3,4],[1,float("inf"),5,6,7],[2,5,float("inf"),9,10],[3,6,9,float("inf"),11],
          [4,7,10,11,float("inf")]]
matrix = np.array(matrix)
print(matrix)
matrix = getNewMatrix(matrix,0,1)
print(matrix)

[[inf  1.  2.  3.  4.]
 [ 1. inf  5.  6.  7.]
 [ 2.  5. inf  9. 10.]
 [ 3.  6.  9. inf 11.]
 [ 4.  7. 10. 11. inf]]
[[inf  9. 10.  3.]
 [ 9. inf 11.  4.]
 [10. 11. inf  5.]
 [ 3.  4.  5. inf]]


In [206]:
#Déclaration globale du dictionnaire de la matrice
proteinDict = {
    'A': 0, 'R': 1, 'N': 2, 'D': 3, 'C': 4, 'Q': 5, 'E': 6, 'G': 7, 'H': 8, 'I': 9, 'L': 10, 'K': 11, 'M': 12, 'F': 13,
    'P': 14,'S': 15, 'T': 16, 'W': 17, 'Y': 18, 'V': 19, 'B': 20, 'Z': 21, 'X': 22, '-': 23
}

def blosumMatrix(input):
    blosum = []
    with open(input) as file:
        blosum = file.readlines()
        blosum = [x.strip() for x in blosum] 
    blosum.pop(0)
    for i in range(len(blosum)):
        blosum[i] = blosum[i].split()
        blosum[i].pop(0)
    return blosum

blosum = blosumMatrix('BLOSUM62.txt')

In [207]:
#Calcul de distance entre deux séquences selon la matrice Blosum62
def distanceBlosum(s1,s2):
    p = 0
    for i in range(0,len(s1)):
        p += int(blosum[proteinDict.get(s1[i])][proteinDict.get(s2[i])])
    qi = 0
    for i in range(0,len(s1)):
        qi += int(blosum[proteinDict.get(s1[i])][proteinDict.get(s1[i])])
    qj = 0
    for i in range(0,len(s2)):
        qj += int(blosum[proteinDict.get(s2[i])][proteinDict.get(s2[i])])
    return 1 - (p / (max(qi,qj)))

# Structure d'arbre

In [208]:
class Tree:
    
    def __init__(self):
        self.root = None
        
    def setRoot(self,node):
        if(self.root == None):
            self.root = node
            node.setRoot()
        else:
            return
        
    def getRoot(self):
        return self.root
    
    #Parcours et affichage postfixe de l'arbre (Internal node = IN)    
    #https://stackoverflow.com/questions/2598437/how-to-implement-a-binary-tree
    def postorderTrav(self):
        if(self.root != None):
            return self._postorderTrav(self.root)
        
    def _postorderTrav(self, node):
        if(node != None):
            leftList = self._postorderTrav(node.leftChild)
            rightList = self._postorderTrav(node.rightChild)
            #print(leftList,rightList)
            name = node.value if(node.value != None) else ("Root" if(node.isRoot) else "IN") 

            return flattenList([leftList,rightList,[name]])
        return [None]
    
    def getLeaves(self):
        if(self.root != None):
            return self._getLeaves(self.root)
        
    def _getLeaves(self, node):
        if(node != None):
            leftList = self._getLeaves(node.leftChild)
            rightList = self._getLeaves(node.rightChild)
            #print(leftList,rightList)
            return flattenList([leftList,rightList,[node.value]])
        return [None]

In [209]:
class Node:
    
    def __init__(self,leftChild,rightChild,value):
        self.leftChild = leftChild
        self.rightChild = rightChild
        self.value = value
        self.isRoot = False
        
    def getLeftChild(self):
        return self.leftChild
    
    def getRightChild(self):
        return self.rightChild
    
    def getValue(self):
        return self.value
    
    def setLeftChild(self,leftChild):
        self.leftChild = leftChild
        
    def setRightChild(self,rightChild):
        self.rightChild = rightChild
        
    def setValue(self,value):
        self.value = value
        
    def isLeaf(self):
        if(self.leftChild == None and self.rightChild == None):
            return True
        else:
            return False
        
    def setRoot(self):
        self.isRoot = True
        
    def isRoot(self):
        return self.isRoot

# Conversion de chaîne Newick

In [210]:
def recursiveNewick(newickSS):
    #NOTE: on considère les chaînes syntaxiquement correctes
    
    #parenthèses ouvertes
    parOpen = 0
    
    #index de départ et de fin de chaque côté de la virgule
    indexStartL = None
    indexEndL = None
    indexStartR = None
    indexEndR = None
    
    #cas où la sous-séquence est un label de feuille
    if(newickSS[0] != "("):
        return Node(None,None,newickSS)
    
    else:
        #passer à travers la chaîne un caractère à la fois
        for i in range(len(newickSS)):
            #début d'un nouveau node
            if(newickSS[i] == "("):
                parOpen += 1
                if(indexStartL == None):
                    indexStartL = i+1
            #séparation des deux enfants
            if(parOpen == 1 and newickSS[i] == ","):
                indexEndL = i-1
                indexStartR = i+1
            #fin du node
            if(newickSS[i] == ")"):
                parOpen -= 1
                if(parOpen == 0):
                    indexEndR = i-1
        
        leftSS = newickSS[indexStartL:indexEndL+1].strip()
        rightSS = newickSS[indexStartR:indexEndR+1].strip()
        
        #rappeler récursivement sur les deux sous-séquences trouvées
        return Node(recursiveNewick(leftSS),recursiveNewick(rightSS),None)

In [211]:
#Test de la fonction de conversion de chaînes
tree = Tree()
tree.setRoot(recursiveNewick("(OR2J3_Humain, ((PCDHA1_Rat, PCDHA1_Souris), PCDHA1_Bonobo));"))
tree.postorderTrav()
#distanceRF(tree)

['OR2J3_Humain',
 'PCDHA1_Rat',
 'PCDHA1_Souris',
 'IN',
 'PCDHA1_Bonobo',
 'IN',
 'Root']

# Calcul de distance RF

In [212]:
def distanceRF(t1,t2):
    #Trouver le nombre de bipartitons non-triviales pour chaque arbre
    b1 = _distanceRFRec(t1.getRoot(),t1.getLeaves())[1]
    b2 = _distanceRFRec(t2.getRoot(),t2.getLeaves())[1]
    
    #print(b1)
    #print(b2)
    
    #parcourir les combinaisons...
    iNot = []
    jNot = []
    for i in range(len(b1)):
        for j in range(len(b2)):
            if((len(list(set(b1[i][0]) - set(b2[j][0]))) == 0 and len(b1[i][0]) == len(b2[j][0]))
               or (len(list(set(b1[i][0]) - set(b2[j][1]))) == 0 and len(b1[i][0]) == len(b2[j][1]))):
                #...et trouver celles qui se retrouvent dans les 2 arbres
                #print(b1[i][0],b2[j][0])
                iNot.append(i)
                jNot.append(j)
    
    #retourner le nombre de bipartitons uniques à chaque arbre
    return len(b1)+len(b2)-len(iNot)-len(jNot)

def _distanceRFRec(node,leaves):
    #structure N retournée récursivement: [[feuilles_observées],[bipartitions_trouvées]]
    
    #N gauche
    left = []
    #N droite
    right = []
    
    #si l'enfant gauche est une feuille, on n'a vu que cette feuille et on n'a aucune bipartition non-triviale
    if(node.leftChild.isLeaf()):
        left = [[node.leftChild.value],[]]
    #sinon, descente récursive
    else:
        left = _distanceRFRec(node.leftChild,leaves)
        
    #même chose à droite
    if(node.rightChild.isLeaf()):
        right = [[node.rightChild.value],[]]
    else:
        right = _distanceRFRec(node.rightChild,leaves)
        
    #unir les bipartitons trouvées des deux côtés
    bipartitions = flattenList([left[1],right[1]])
    
    #valeur permettant de couper une seule fois (ou pas du tout) à la racine
    oneCut = float("inf")
    if(node.isRoot):
        #si les deux sous-arbres de la racine ont plus d'une feuille,
        #on peut la couper une fois 
        if(len(left[0]) > 1 and len(right[0]) > 1):
            oneCut = 1
        #sinon, on ne peut pas couper car trivial
        else:
            oneCut = 0
        
    #si non-trivial à gauche et permis de couper,
    #on crée une nouvelle bipartiton
    if(len(left[0]) > 1 and oneCut > 0):
        partition1 = left[0]
        partition2 = (list(set(leaves) - set(partition1)))
        bipartitions.append([partition1,partition2])
        oneCut -= 1

    #même chose à droite
    if(len(right[0]) > 1 and oneCut > 0):
        partition1 = right[0]
        partition2 = (list(set(leaves) - set(partition1)))
        bipartitions.append([partition1,partition2])
     
    #unir les feuilles visitées à gauche et à droite
    leavesDown = flattenList([left[0],right[0]])
    
    return [leavesDown,bipartitions]

# Neighbour joining

In [213]:
def neighbourJoining(matrix,labels):
    tree = Tree()
    
    L =[]
    
    #Créer les feuilles
    for i in range(len(matrix)):
        L.append(Node(None,None,labels[i]))
            
    while(len(L) > 2):
        #Trouver paire minimale
        minIndex = np.argwhere(matrix == np.min(matrix))[0]
        #print(minIndex)
        
        indexI = minIndex[0]
        indexJ = minIndex[1]
        
        #noeuds associés à l'index minimal
        i = L[indexI]
        j = L[indexJ]
        k = Node(i,j,None) #nouveau noeud ayant i et j comme enfants
               
        #enlever i et j, ajouter k à L
        del L[indexI]
        del L[indexJ-(1 if (indexI < indexJ) else 0)]
        L.append(k)

        #recalculer la matricce de distances
        matrix = getNewMatrix(matrix,indexI,indexJ)
             
    #Transformer la dernière arrête en racine
    root = Node(L[0],L[1],None)
    tree.setRoot(root)
    
    return tree

In [214]:
#Test du neighbour joining
matrix = [[float("inf"),1,2,3,4],[1,float("inf"),5,6,7],[2,5,float("inf"),9,10],[3,6,9,float("inf"),11],
          [4,7,10,11,float("inf")]]
matrix = np.array(matrix)
labels = ["A","B","C","D","E"]

tree = neighbourJoining(matrix,labels)
tree.postorderTrav()

['E', 'D', 'C', 'A', 'B', 'IN', 'IN', 'IN', 'Root']

# Parsing des séquences d'entrée

In [215]:
#Créer la 1ere matrice de distances
def distMatrix(sequences):
    distMatrix = np.diag(np.full((len(sequences[0]),len(sequences[0])),float("inf")).diagonal())
    for i in range(0, len(sequences[0])): 
        for j in range(i+1, len(sequences[0])):
            distMatrix[i][j] = distanceBlosum(sequences[1][i],sequences[1][j])
            distMatrix[j][i] = distMatrix[i][j]
    return distMatrix

#Parsing de l'alignement sur fichier d'input
def readProteinFile(input):
    inputAsList = []
    with open(input) as file:
        protein = file.readlines()
    for i in range (0, len(protein)):
        inputAsList.append(protein[i][:-1])
    concatenation = ""
    labels = []
    protein = []
    for i in range (0, len(inputAsList)):
        if (i % 8 == 7) :
            protein.append(concatenation)
            concatenation = ""
        elif (i % 8 == 0) :
            labels.append(inputAsList[i][1:])
        else:
            concatenation += inputAsList[i]
    return [labels, protein]          

# Comparaison entre les chaînes de Newick et l'alignement

In [216]:
reads = readProteinFile('proteines.fa')
labels = reads[0]
matrix = distMatrix(reads)

treeS = neighbourJoining(matrix,labels)

print(treeS.postorderTrav())

readsFile = open("arbres.nw", "r")
readsLines = readsFile.readlines()
readsFile.close()

print("\nDistance RF")
for i in range(len(readsLines)):
    treeN = Tree()
    treeN.setRoot(recursiveNewick(readsLines[i]))
    #print(treeN.postorderTrav())
    print("Newick " + str(i) + ": " + str(distanceRF(treeS,treeN)))

['OR2J3_Humain', 'PCDHA1_Humain', 'PCDHA1_Bonobo', 'IN', 'PCDHA1_Rat', 'PCDHA1_Souris', 'IN', 'IN', 'Root']

Distance RF
Newick 0: 2
Newick 1: 4
Newick 2: 4
Newick 3: 0
Newick 4: 4


Bref, la 4e chaîne de Newick construit l'arbre ayant la plus grande similarité avec l'alignement multiple en entrée.